<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/_Code_for_training_a_logistic_regression_model_and_visualizing_SHAP_values_and_VGG16_fine_tuning_project_using_PyTorch_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

In [ ]:
pip install shap torch scikit-learn numpy

In [ ]:
pip install --upgrade shap

In [ ]:
import shap
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Create synthetic data
X, y = make_classification(n_samples=500, n_features=20, n_classes=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f"Model accuracy: {accuracy:.2f}")

# Initialize SHAP explainer
explainer = shap.Explainer(model, X_train)

# Generate SHAP values for the test set
shap_values = explainer(X_test)

# Check the number of samples and features for SHAP values
num_samples = X_test.shape[0]
num_features = X_test.shape[1]

# Verify SHAP values shape
print(f"SHAP values shape: {shap_values.shape}")

# Ensure the SHAP values' shape matches the input data
if shap_values.values.shape == (num_samples, num_features):
    print("SHAP values match the input data shape.")

    # Use SHAP values for visualization
    shap.summary_plot(shap_values, X_test, feature_names=[f"Feature {i}" for i in range(X_test.shape[1])])

    # Feature names
    feature_names = [f"Feature {i}" for i in range(X_test.shape[1])]

    # Force plot for the first sample
    shap.force_plot(shap_values[0].base_values, shap_values.values[0], X_test[0], feature_names=feature_names, matplotlib=True)
else:
    print("The SHAP values' shape does not match the input data.")

In [ ]:
import shap
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'model' is your trained model and 'X_test' is your test data

# Create a SHAP explainer for the model
explainer = shap.Explainer(model, X_test)

# Calculate SHAP values for all samples
shap_values = explainer(X_test)

# Check the shape of SHAP values and X_test
print("SHAP values shape:", shap_values.shape)  # Expected (100, 20)
print("X_test shape:", X_test.shape)  # Expected (100, 20)

# Extract SHAP values for class 0 for each sample (all features)
# Now we capture all feature SHAP values for class 0 for each sample
shap_values_class_0 = np.array([shap_values[i].values for i in range(len(shap_values))])

# Check the shape of the extracted SHAP values for class 0
print(f"SHAP values for class 0 shape (after collection): {shap_values_class_0.shape}")

# Ensure the SHAP values for class 0 have the shape (100, 20)
# Since each sample should have SHAP values for all features (20 features for 100 samples)
assert shap_values_class_0.shape == (X_test.shape[0], X_test.shape[1]), \
    f"Expected shape (100, 20), but got {shap_values_class_0.shape}"

# Create feature names (optional: customize if you have actual feature names)
feature_names = [f"Feature {i}" for i in range(X_test.shape[1])]

# Generate the SHAP summary plot for class 0
shap.summary_plot(shap_values_class_0, X_test, feature_names=feature_names)

# Optionally, generate the bar plot for feature importance
shap.summary_plot(shap_values_class_0, X_test, plot_type="bar", feature_names=feature_names)

# Show the plots
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
# Ensure that the number of informative, redundant, and repeated features is less than total features
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(5, 8)  # Input layer with 5 features
        self.fc2 = nn.Linear(8, 4)  # Hidden layer
        self.fc3 = nn.Linear(4, 2)  # Output layer (2 classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation function
        x = torch.relu(self.fc2(x))  # ReLU activation function
        x = self.fc3(x)  # Output layer (no activation function here)
        return x

# 6. Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis
# Wrap the model's forward function to accept NumPy arrays for SHAP compatibility
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return the raw output (logits)

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Use a small subset for background data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values for the first sample
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
# Ensure that the number of informative, redundant, and repeated features is less than total features
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(5, 8)  # Input layer with 5 features
        self.fc2 = nn.Linear(8, 4)  # Hidden layer
        self.fc3 = nn.Linear(4, 2)  # Output layer (2 classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation function
        x = torch.relu(self.fc2(x))  # ReLU activation function
        x = self.fc3(x)  # Output layer (no activation function here)
        return x

# 6. Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis
# Wrap the model's forward function to accept NumPy arrays for SHAP compatibility
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return the raw output (logits)

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Use a small subset for background data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values for the first sample
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
# Ensuring number of informative features is less than the total number of features
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(5, 8)  # Input layer with 5 features
        self.fc2 = nn.Linear(8, 4)  # Hidden layer
        self.fc3 = nn.Linear(4, 2)  # Output layer (2 classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation function
        x = torch.relu(self.fc2(x))  # ReLU activation function
        x = self.fc3(x)  # Output layer (no activation function here)
        return x

# 6. Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis
# Wrap the model's forward function to accept NumPy arrays for SHAP compatibility
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return the raw output (logits)

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Use a small subset for background data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values for the first sample
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
# Ensuring number of informative features is less than the total number of features
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(5, 8)  # Input layer with 5 features
        self.fc2 = nn.Linear(8, 4)  # Hidden layer
        self.fc3 = nn.Linear(4, 2)  # Output layer (2 classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation function
        x = torch.relu(self.fc2(x))  # ReLU activation function
        x = self.fc3(x)  # Output layer (no activation function here)
        return x

# 6. Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis
# Wrap the model's forward function to accept NumPy arrays for SHAP compatibility
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return the raw output (logits)

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Use a small subset for background data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values for the first sample
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(5, 8)  # Input layer with 5 features
        self.fc2 = nn.Linear(8, 4)  # Hidden layer
        self.fc3 = nn.Linear(4, 2)  # Output layer (2 classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation function
        x = torch.relu(self.fc2(x))  # ReLU activation function
        x = self.fc3(x)  # Output layer (no activation function here)
        return x

# 6. Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis
# Wrap the model's forward function to accept NumPy arrays for SHAP compatibility
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return the raw output (logits)

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Use a small subset for background data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values for the first sample
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(5, 8)  # Input layer with 5 features
        self.fc2 = nn.Linear(8, 4)  # Hidden layer
        self.fc3 = nn.Linear(4, 2)  # Output layer (2 classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation function
        x = torch.relu(self.fc2(x))  # ReLU activation function
        x = self.fc3(x)  # Output layer (no activation function here)
        return x

# 6. Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis
# Wrap the model's forward function to accept NumPy arrays for SHAP compatibility
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return the raw output (logits)

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Use a small subset for background data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values for the first sample
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 16)  # Input layer with 5 features, 16 neurons
        self.bn1 = nn.BatchNorm1d(16)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(16, 8)  # Hidden layer with 8 neurons
        self.bn2 = nn.BatchNorm1d(8)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(8, 2)   # Output layer (2 classes)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer to prevent overfitting

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU + BatchNorm + FC1
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU + BatchNorm + FC2
        x = self.dropout(x)  # Apply dropout
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Train the model
num_epochs = 20  # Increased epochs for better convergence
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis
# Wrap the model's forward function to accept NumPy arrays for SHAP compatibility
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return the raw output (logits)

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Use a small subset for background data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values for the first sample
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # Increased neurons in the first layer
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Hidden layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer (2 classes)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer to prevent overfitting

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU + BatchNorm + FC1
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU + BatchNorm + FC2
        x = self.dropout(x)  # Apply dropout
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Number of epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis
# Wrap the model's forward function to accept NumPy arrays for SHAP compatibility
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return the raw output (logits)

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Use a small subset for background data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values for the first sample
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # Increased neurons in the first layer
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Hidden layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer (2 classes)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer to prevent overfitting

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU + BatchNorm + FC1
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU + BatchNorm + FC2
        x = self.dropout(x)  # Apply dropout
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Number of epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis
# Wrap the model's forward function to accept NumPy arrays for SHAP compatibility
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return the raw output (logits)

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Use a small subset for background data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values for the first sample
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with Batch Normalization and Dropout
class EnhancedNN(nn.Module):
    def __init__(self):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 32)  # First layer with 32 neurons
        self.bn1 = nn.BatchNorm1d(32)  # Batch Normalization after first layer
        self.fc2 = nn.Linear(32, 16)  # Second layer with 16 neurons
        self.bn2 = nn.BatchNorm1d(16)  # Batch Normalization after second layer
        self.fc3 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% drop rate

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # ReLU activation + BatchNorm + First layer
        x = torch.relu(self.bn2(self.fc2(x)))  # ReLU activation + BatchNorm + Second layer
        x = self.dropout(x)  # Dropout applied after hidden layers
        x = self.fc3(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = EnhancedNN()
criterion = nn.CrossEntropyLoss()  # CrossEntropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# 7. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 8. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 9. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with advanced features
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 64)  # First layer with 64 neurons
        self.fc2 = nn.Linear(64, 32)  # Second layer with 32 neurons
        self.fc3 = nn.Linear(32, 16)  # Third layer with 16 neurons
        self.fc4 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.bn1 = nn.BatchNorm1d(64)  # Batch Normalization after first hidden layer
        self.bn2 = nn.BatchNorm1d(32)  # Batch Normalization after second hidden layer
        self.bn3 = nn.BatchNorm1d(16)  # Batch Normalization after third hidden layer
        self.dropout = nn.Dropout(p=0.5)  # Dropout with 50% rate

        # Weight Initialization (He Initialization for better convergence)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        nn.init.xavier_normal_(self.fc4.weight)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # First layer with ReLU activation and BatchNorm
        x = torch.relu(self.bn2(self.fc2(x)))  # Second layer with ReLU activation and BatchNorm
        x = torch.relu(self.bn3(self.fc3(x)))  # Third layer with ReLU activation and BatchNorm
        x = self.dropout(x)  # Apply dropout to prevent overfitting
        x = self.fc4(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = AdvancedNN()
criterion = nn.CrossEntropyLoss()  # For binary classification, use CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# 7. Learning Rate Scheduler (ReduceLROnPlateau)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)

# 8. Train the model
num_epochs = 20  # Training for 20 epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Adjust learning rate based on validation loss (scheduler)
    scheduler.step(loss)

    if (epoch + 1) % 2 == 0:  # Print loss every 2 epochs
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 9. Evaluate the model
model.eval()  # Switch model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get predicted class labels
    correct = (predicted == y_test_tensor).sum().item()  # Compare predictions to actual labels
    accuracy = correct / y_test_tensor.size(0) * 100  # Calculate accuracy
    print(f"Test Accuracy: {accuracy:.2f}%")

# 10. SHAP analysis for model interpretability
# Define a function to wrap the model for SHAP analysis (works with NumPy input)
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])  # Using a small subset of training data

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np
import copy

# 1. Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)

# 2. Split the dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 3. Normalize the dataset using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# 4. Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 5. Define the neural network model with advanced features
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 64)  # First layer with 64 neurons
        self.fc2 = nn.Linear(64, 32)  # Second layer with 32 neurons
        self.fc3 = nn.Linear(32, 16)  # Third layer with 16 neurons
        self.fc4 = nn.Linear(16, 2)   # Output layer with 2 neurons (for binary classification)
        self.bn1 = nn.BatchNorm1d(64)  # Batch Normalization after first hidden layer
        self.bn2 = nn.BatchNorm1d(32)  # Batch Normalization after second hidden layer
        self.bn3 = nn.BatchNorm1d(16)  # Batch Normalization after third hidden layer
        self.dropout = nn.Dropout(p=0.5)  # Dropout with 50% rate

        # Weight Initialization (He Initialization for better convergence)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        nn.init.xavier_normal_(self.fc4.weight)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # First layer with ReLU activation and BatchNorm
        x = torch.relu(self.bn2(self.fc2(x)))  # Second layer with ReLU activation and BatchNorm
        x = torch.relu(self.bn3(self.fc3(x)))  # Third layer with ReLU activation and BatchNorm
        x = self.dropout(x)  # Apply dropout to prevent overfitting
        x = self.fc4(x)  # Output layer
        return x

# 6. Initialize the model, loss function, and optimizer
model = AdvancedNN()
criterion = nn.CrossEntropyLoss()  # For binary classification, use CrossEntropyLoss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# 7. Learning Rate Scheduler (ReduceLROnPlateau)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

# 8. Training and validation loop with model saving based on validation accuracy
num_epochs = 20
best_val_accuracy = 0.0
best_model = None

for epoch in range(num_epochs):
    # Training phase
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    scheduler.step(loss)  # Scheduler based on training loss

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        _, val_predicted = torch.max(val_outputs, 1)
        val_correct = (val_predicted == y_val_tensor).sum().item()
        val_accuracy = val_correct / y_val_tensor.size(0) * 100

    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = copy.deepcopy(model.state_dict())

    # Print progress every 2 epochs
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Load best model
model.load_state_dict(best_model)

# 9. Evaluate the model on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, test_predicted = torch.max(test_outputs, 1)
    test_correct = (test_predicted == y_test_tensor).sum().item()
    test_accuracy = test_correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {test_accuracy:.2f}%")

# 10. SHAP analysis for model interpretability
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()  # Return raw logits for SHAP

# Initialize SHAP KernelExplainer
explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])

# Calculate SHAP values for the first 10 test samples
shap_values = explainer.shap_values(X_test[:10])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np
import copy

# 1. Dataset generation
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 2. Advanced model
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 2)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(16)
        self.dropout = nn.Dropout(p=0.5)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        nn.init.xavier_normal_(self.fc4.weight)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# 3. Initialization and configurations
model = AdvancedNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([
    {'params': model.fc1.parameters(), 'lr': 0.001},
    {'params': model.fc2.parameters(), 'lr': 0.0005},
    {'params': model.fc3.parameters(), 'lr': 0.0001},
    {'params': model.fc4.parameters(), 'lr': 0.0001}
])

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
clip_value = 1.0  # For gradient clipping
early_stopping_patience = 5  # Early stopping if no improvement in 5 epochs

# 4. Training loop with early stopping and gradient clipping
num_epochs = 20
best_val_accuracy = 0.0
best_model = None
no_improvement_epochs = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
    optimizer.step()
    scheduler.step(loss)

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        _, val_predicted = torch.max(val_outputs, 1)
        val_correct = (val_predicted == y_val_tensor).sum().item()
        val_accuracy = val_correct / y_val_tensor.size(0) * 100

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = copy.deepcopy(model.state_dict())
        no_improvement_epochs = 0  # Reset counter if improved
    else:
        no_improvement_epochs += 1

    # Early stopping condition
    if no_improvement_epochs >= early_stopping_patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Load best model based on validation accuracy
model.load_state_dict(best_model)

# 5. Test evaluation
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, test_predicted = torch.max(test_outputs, 1)
    test_correct = (test_predicted == y_test_tensor).sum().item()
    test_accuracy = test_correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {test_accuracy:.2f}%")

# 6. SHAP Analysis
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()

explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])
shap_values = explainer.shap_values(X_test[:10])

# Plot SHAP summary
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np
import copy

# 1. Dataset generation
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 2. Advanced model with higher dropout and weight decay
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 2)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(16)
        self.dropout = nn.Dropout(p=0.6)  # Increased dropout
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        nn.init.xavier_normal_(self.fc4.weight)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

# 3. Initialization and configurations
model = AdvancedNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Added weight decay
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
clip_value = 1.0
early_stopping_patience = 5

# 4. Training loop with gradient clipping and cosine annealing scheduler
num_epochs = 20
best_val_accuracy = 0.0
best_model = None
no_improvement_epochs = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
    optimizer.step()
    scheduler.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        _, val_predicted = torch.max(val_outputs, 1)
        val_correct = (val_predicted == y_val_tensor).sum().item()
        val_accuracy = val_correct / y_val_tensor.size(0) * 100

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = copy.deepcopy(model.state_dict())
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    # Early stopping condition
    if no_improvement_epochs >= early_stopping_patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Load best model based on validation accuracy
model.load_state_dict(best_model)

# 5. Test evaluation
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, test_predicted = torch.max(test_outputs, 1)
    test_correct = (test_predicted == y_test_tensor).sum().item()
    test_accuracy = test_correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {test_accuracy:.2f}%")

# 6. SHAP Analysis
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()

explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])
shap_values = explainer.shap_values(X_test[:10])

# Plot SHAP summary
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np
import copy

# 1. Dataset generation
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 2. Optimized model with layer normalization and warm-up
class OptimizedNN(nn.Module):
    def __init__(self):
        super(OptimizedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 12)
        self.fc4 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropout = nn.Dropout(p=0.5)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        nn.init.xavier_normal_(self.fc4.weight)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.ln2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.ln3(self.fc3(x)))
        x = self.fc4(x)
        return x

# 3. Initialization and configurations
model = OptimizedNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
clip_value = 1.0
early_stopping_patience = 5

# Warm-up scheduler for the first few epochs
warmup_epochs = 5
warmup_scheduler = lambda epoch: (epoch + 1) / warmup_epochs if epoch < warmup_epochs else 1.0

# 4. Training loop with gradient clipping, warm-up, and scheduler
num_epochs = 20
best_val_accuracy = 0.0
best_model = None
no_improvement_epochs = 0

for epoch in range(num_epochs):
    model.train()
    optimizer.param_groups[0]['lr'] = warmup_scheduler(epoch) * optimizer.defaults['lr']
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
    optimizer.step()
    if epoch >= warmup_epochs:
        scheduler.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        _, val_predicted = torch.max(val_outputs, 1)
        val_correct = (val_predicted == y_val_tensor).sum().item()
        val_accuracy = val_correct / y_val_tensor.size(0) * 100

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = copy.deepcopy(model.state_dict())
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    # Early stopping condition
    if no_improvement_epochs >= early_stopping_patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Load best model based on validation accuracy
model.load_state_dict(best_model)

# 5. Test evaluation
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, test_predicted = torch.max(test_outputs, 1)
    test_correct = (test_predicted == y_test_tensor).sum().item()
    test_accuracy = test_correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {test_accuracy:.2f}%")

# 6. SHAP Analysis
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()

explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])
shap_values = explainer.shap_values(X_test[:10])

# Plot SHAP summary
shap.initjs()
shap.summary_plot(shap_values, X_test[:10])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np
import copy

# 1. Dataset generation
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 2. Optimized model with advanced normalization and dropout layers
class OptimizedNN(nn.Module):
    def __init__(self):
        super(OptimizedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 12)
        self.fc4 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropout = nn.Dropout(p=0.4)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        nn.init.xavier_normal_(self.fc4.weight)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.ln2(self.fc2(x)))
        x = self.dropout(x)
        x = torch.relu(self.ln3(self.fc3(x)))
        x = self.fc4(x)
        return x

# 3. Training configuration with CLR and patience restart
model = OptimizedNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.002, step_size_up=10, mode="triangular")
clip_value = 1.0
early_stopping_patience = 4

num_epochs = 20
best_val_accuracy = 0.0
best_model = None
no_improvement_epochs = 0

# 4. Training loop with cyclical LR and early stopping with patience restart
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
    optimizer.step()
    scheduler.step()  # Cyclical LR adjustment

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        _, val_predicted = torch.max(val_outputs, 1)
        val_correct = (val_predicted == y_val_tensor).sum().item()
        val_accuracy = val_correct / y_val_tensor.size(0) * 100

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = copy.deepcopy(model.state_dict())
        no_improvement_epochs = 0  # Reset if improvement
    else:
        no_improvement_epochs += 1

    # Early stopping condition with patience restart
    if no_improvement_epochs >= early_stopping_patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Load best model based on validation accuracy
model.load_state_dict(best_model)

# 5. Test evaluation
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, test_predicted = torch.max(test_outputs, 1)
    test_correct = (test_predicted == y_test_tensor).sum().item()
    test_accuracy = test_correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {test_accuracy:.2f}%")

# 6. SHAP Analysis with focus on high-impact features
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()

explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])
shap_values = explainer.shap_values(X_test[:10])

# Plot SHAP summary focusing on most impactful features
shap.initjs()
shap.summary_plot(shap_values, X_test[:10], max_display=5)  # Show top 5 features for impact clarity

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import shap
import numpy as np
import copy

# 1. Dataset generation
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, n_classes=2, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 2. Optimized model with AGC and Dropconnect regularization
class OptimizedNN(nn.Module):
    def __init__(self):
        super(OptimizedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 12)
        self.fc4 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.2)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        nn.init.xavier_normal_(self.fc4.weight)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))
        x = self.dropconnect(x)
        x = torch.relu(self.ln3(self.fc3(x)))
        x = self.fc4(x)
        return x

# 3. Training configuration with AGC, SGD with Momentum, and CLR
model = OptimizedNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01, step_size_up=10, mode="triangular")
early_stopping_patience = 4
num_epochs = 20
best_val_accuracy = 0.0
best_model = None
no_improvement_epochs = 0

def adaptive_gradient_clipping(params, clip_value=0.01):
    with torch.no_grad():
        for param in params:
            if param.grad is not None:
                grad_norm = param.grad.norm()
                param_norm = param.norm()
                max_norm = clip_value * max(param_norm, 1e-6)
                if grad_norm > max_norm:
                    param.grad.mul_(max_norm / grad_norm)

# 4. Training loop with AGC and early stopping with patience restart
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    adaptive_gradient_clipping(model.parameters())
    optimizer.step()
    scheduler.step()  # Cyclical LR adjustment

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        _, val_predicted = torch.max(val_outputs, 1)
        val_correct = (val_predicted == y_val_tensor).sum().item()
        val_accuracy = val_correct / y_val_tensor.size(0) * 100

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = copy.deepcopy(model.state_dict())
        no_improvement_epochs = 0  # Reset if improvement
    else:
        no_improvement_epochs += 1

    # Early stopping condition with patience restart
    if no_improvement_epochs >= early_stopping_patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Load best model based on validation accuracy
model.load_state_dict(best_model)

# 5. Test evaluation
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, test_predicted = torch.max(test_outputs, 1)
    test_correct = (test_predicted == y_test_tensor).sum().item()
    test_accuracy = test_correct / y_test_tensor.size(0) * 100
    print(f"Test Accuracy: {test_accuracy:.2f}%")

# 6. SHAP Analysis with focus on high-impact features
def model_with_numpy_input(X):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    outputs = model(X_tensor)
    return outputs.detach().numpy()

explainer = shap.KernelExplainer(model_with_numpy_input, X_train[:100])
shap_values = explainer.shap_values(X_test[:10])

# Plot SHAP summary focusing on most impactful features
shap.initjs()
shap.summary_plot(shap_values, X_test[:10], max_display=5)  # Show top 5 features for impact clarity

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.datasets import make_classification
import numpy as np
from tqdm import tqdm
import os

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Define Mixup function
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Define the Advanced Neural Network without in-place operations
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.3)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x_res = x  # Skip connection
        x = torch.relu(self.fc3(x))  # Non-in-place ReLU for skip connection
        x = x + x_res  # Add skip connection
        x = torch.relu(self.ln3(self.fc4(x)))  # Non-in-place ReLU
        x = self.fc5(x)
        return x

# Hyperparameters and configurations
batch_size = 32
learning_rate = 0.001
num_epochs = 20
patience = 5  # For early stopping
accumulation_steps = 2
model_save_path = 'best_model.pth'  # Path to save the best model

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, optimizer, and learning rate scheduler
model = AdvancedNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Apply label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

# Training and Validation Loop with Early Stopping and Gradient Accumulation
best_val_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0

    # Track loss over training iterations for smoothing
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss = loss / accumulation_steps  # Adjust loss for accumulation
        loss.backward()  # Accumulate gradients

        # Update weights every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()

    # Track average training loss for the epoch
    train_losses.append(epoch_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_accuracy = val_correct / len(val_dataset) * 100
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}, Loss: {train_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Learning rate scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
        # Save the model with the best validation accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Saved model with validation accuracy {best_val_accuracy:.2f}%")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

# Load the best model
model.load_state_dict(torch.load(model_save_path))

# Testing Phase
model.eval()
test_correct = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == targets).sum().item()

test_accuracy = test_correct / len(test_dataset) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Optionally, plot the training and validation losses/accuracies
import matplotlib.pyplot as plt

# Plot the loss curve
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.datasets import make_classification
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Define Mixup function
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Define the Advanced Neural Network without in-place operations
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.3)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x_res = x  # Skip connection
        x = torch.relu(self.fc3(x))  # Non-in-place ReLU for skip connection
        x = x + x_res  # Add skip connection
        x = torch.relu(self.ln3(self.fc4(x)))  # Non-in-place ReLU
        x = self.fc5(x)
        return x

# Hyperparameters and configurations
batch_size = 32
learning_rate = 0.001
num_epochs = 20
patience = 5  # For early stopping
accumulation_steps = 2
model_save_path = 'best_model.pth'  # Path to save the best model

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, optimizer, and learning rate scheduler
model = AdvancedNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Apply label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

# Training and Validation Loop with Early Stopping and Gradient Accumulation
best_val_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0

    # Track loss over training iterations for smoothing
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss = loss / accumulation_steps  # Adjust loss for accumulation
        loss.backward()  # Accumulate gradients

        # Update weights every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()

    # Track average training loss for the epoch
    train_losses.append(epoch_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_accuracy = val_correct / len(val_dataset) * 100
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}, Loss: {train_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Learning rate scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
        # Save the model with the best validation accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Saved model with validation accuracy {best_val_accuracy:.2f}%")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

# Load the best model (with weights_only=True to avoid security warning)
model.load_state_dict(torch.load(model_save_path, weights_only=True))

# Testing Phase
model.eval()
test_correct = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == targets).sum().item()

test_accuracy = test_correct / len(test_dataset) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Optionally, plot the training and validation losses/accuracies
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.datasets import make_classification
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Define Mixup function
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Define the Advanced Neural Network without in-place operations
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.3)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x_res = x  # Skip connection
        x = torch.relu(self.fc3(x))  # Non-in-place ReLU for skip connection
        x = x + x_res  # Add skip connection
        x = torch.relu(self.ln3(self.fc4(x)))  # Non-in-place ReLU
        x = self.fc5(x)
        return x

# Hyperparameters and configurations
batch_size = 32
learning_rate = 0.001
num_epochs = 20
patience = 3  # Early stopping patience
accumulation_steps = 2
model_save_path = 'best_model.pth'  # Path to save the best model

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, optimizer, and learning rate scheduler
model = AdvancedNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Apply label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

# Training and Validation Loop with Early Stopping and Gradient Accumulation
best_val_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0

    # Track loss over training iterations for smoothing
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss = loss / accumulation_steps  # Adjust loss for accumulation
        loss.backward()  # Accumulate gradients

        # Update weights every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()

    # Track average training loss for the epoch
    train_losses.append(epoch_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_accuracy = val_correct / len(val_dataset) * 100
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}, Loss: {train_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Learning rate scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
        # Save the model with the best validation accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Saved model with validation accuracy {best_val_accuracy:.2f}%")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

# Load the best model (with weights_only=True to avoid security warning)
model.load_state_dict(torch.load(model_save_path, weights_only=True))

# Testing Phase
model.eval()
test_correct = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == targets).sum().item()

test_accuracy = test_correct / len(test_dataset) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Optionally, plot the training and validation losses/accuracies
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.datasets import make_classification
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Define Mixup function
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Define the Advanced Neural Network without in-place operations
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.3)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x_res = x  # Skip connection
        x = torch.relu(self.fc3(x))  # Non-in-place ReLU for skip connection
        x = x + x_res  # Add skip connection
        x = torch.relu(self.ln3(self.fc4(x)))  # Non-in-place ReLU
        x = self.fc5(x)
        return x

# Hyperparameters and configurations
batch_size = 32
learning_rate = 0.001
num_epochs = 20
patience = 3  # Early stopping patience
accumulation_steps = 2
model_save_path = 'best_model.pth'  # Path to save the best model

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, optimizer, and learning rate scheduler
model = AdvancedNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Apply label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

# Training and Validation Loop with Early Stopping and Gradient Accumulation
best_val_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0

    # Track loss over training iterations for smoothing
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss = loss / accumulation_steps  # Adjust loss for accumulation
        loss.backward()  # Accumulate gradients

        # Update weights every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()

    # Track average training loss for the epoch
    train_losses.append(epoch_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_accuracy = val_correct / len(val_dataset) * 100
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}, Loss: {train_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Learning rate scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
        # Save the model with the best validation accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Saved model with validation accuracy {best_val_accuracy:.2f}%")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

# Load the best model (with weights_only=True to avoid security warning)
model.load_state_dict(torch.load(model_save_path, weights_only=True))

# Testing Phase
model.eval()
test_correct = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == targets).sum().item()

test_accuracy = test_correct / len(test_dataset) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Optionally, plot the training and validation losses/accuracies
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.datasets import make_classification
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Define Mixup function
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Define the Advanced Neural Network without in-place operations
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.3)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x_res = x  # Skip connection
        x = torch.relu(self.fc3(x))  # Non-in-place ReLU for skip connection
        x = x + x_res  # Add skip connection
        x = torch.relu(self.ln3(self.fc4(x)))  # Non-in-place ReLU
        x = self.fc5(x)
        return x

# Hyperparameters and configurations
batch_size = 32
learning_rate = 0.001
num_epochs = 20
patience = 3  # Early stopping patience
accumulation_steps = 2
model_save_path = 'best_model.pth'  # Path to save the best model

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, optimizer, and learning rate scheduler
model = AdvancedNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Apply label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

# Training and Validation Loop with Early Stopping and Gradient Accumulation
best_val_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0

    # Track loss over training iterations for smoothing
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss = loss / accumulation_steps  # Adjust loss for accumulation
        loss.backward()  # Accumulate gradients

        # Update weights every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()

    # Track average training loss for the epoch
    train_losses.append(epoch_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_accuracy = val_correct / len(val_dataset) * 100
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}, Loss: {train_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Learning rate scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
        # Save the model with the best validation accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Saved model with validation accuracy {best_val_accuracy:.2f}%")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

# Load the best model (with weights_only=True to avoid security warning)
model.load_state_dict(torch.load(model_save_path, weights_only=True))

# Testing Phase
model.eval()
test_correct = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == targets).sum().item()

test_accuracy = test_correct / len(test_dataset) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Optionally, plot the training and validation losses/accuracies
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.datasets import make_classification
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Define Mixup function
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Define the Advanced Neural Network without in-place operations
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.3)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x_res = x  # Skip connection
        x = torch.relu(self.fc3(x))  # Non-in-place ReLU for skip connection
        x = x + x_res  # Add skip connection
        x = torch.relu(self.ln3(self.fc4(x)))  # Non-in-place ReLU
        x = self.fc5(x)
        return x

# Hyperparameters and configurations
batch_size = 32
learning_rate = 0.001
num_epochs = 20
patience = 3  # Early stopping patience
accumulation_steps = 2
model_save_path = 'best_model.pth'  # Path to save the best model

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, optimizer, and learning rate scheduler
model = AdvancedNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Apply label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

# Training and Validation Loop with Early Stopping and Gradient Accumulation
best_val_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0

    # Track loss over training iterations for smoothing
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss = loss / accumulation_steps  # Adjust loss for accumulation
        loss.backward()  # Accumulate gradients

        # Update weights every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()

    # Track average training loss for the epoch
    train_losses.append(epoch_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_accuracy = val_correct / len(val_dataset) * 100
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}, Loss: {train_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Learning rate scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
        # Save the model with the best validation accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Saved model with validation accuracy {best_val_accuracy:.2f}%")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

# Load the best model (with weights_only=True to avoid security warning)
model.load_state_dict(torch.load(model_save_path, weights_only=True))

# Testing Phase
model.eval()
test_correct = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == targets).sum().item()

test_accuracy = test_correct / len(test_dataset) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Optionally, plot the training and validation losses/accuracies
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.datasets import make_classification
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Define Mixup function
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Define the Advanced Neural Network without in-place operations
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.3)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x_res = x  # Skip connection
        x = torch.relu(self.fc3(x))  # Non-in-place ReLU for skip connection
        x = x + x_res  # Add skip connection
        x = torch.relu(self.ln3(self.fc4(x)))  # Non-in-place ReLU
        x = self.fc5(x)
        return x

# Hyperparameters and configurations
batch_size = 32
learning_rate = 0.001
num_epochs = 20
patience = 3  # Early stopping patience
accumulation_steps = 2
model_save_path = 'best_model.pth'  # Path to save the best model

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, optimizer, and learning rate scheduler
model = AdvancedNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Apply label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

# Training and Validation Loop with Early Stopping and Gradient Accumulation
best_val_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0

    # Track loss over training iterations for smoothing
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss = loss / accumulation_steps  # Adjust loss for accumulation
        loss.backward()  # Accumulate gradients

        # Update weights every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()

    # Track average training loss for the epoch
    train_losses.append(epoch_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_accuracy = val_correct / len(val_dataset) * 100
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}, Loss: {train_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Learning rate scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
        # Save the model with the best validation accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Saved model with validation accuracy {best_val_accuracy:.2f}%")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

# Load the best model (with weights_only=True to avoid security warning)
model.load_state_dict(torch.load(model_save_path, weights_only=True))

# Testing Phase
model.eval()
test_correct = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == targets).sum().item()

test_accuracy = test_correct / len(test_dataset) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Optionally, plot the training and validation losses/accuracies
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.datasets import make_classification
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Define Mixup function
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Define the Advanced Neural Network without in-place operations
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.3)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x_res = x  # Skip connection
        x = torch.relu(self.fc3(x))  # Non-in-place ReLU for skip connection
        x = x + x_res  # Add skip connection
        x = torch.relu(self.ln3(self.fc4(x)))  # Non-in-place ReLU
        x = self.fc5(x)
        return x

# Hyperparameters and configurations
batch_size = 32
learning_rate = 0.001
num_epochs = 20
patience = 3  # Early stopping patience
accumulation_steps = 2
model_save_path = 'best_model.pth'  # Path to save the best model

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, optimizer, and learning rate scheduler
model = AdvancedNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Apply label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

# Training and Validation Loop with Early Stopping and Gradient Accumulation
best_val_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0

    # Track loss over training iterations for smoothing
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss = loss / accumulation_steps  # Adjust loss for accumulation
        loss.backward()  # Accumulate gradients

        # Update weights every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()

    # Track average training loss for the epoch
    train_losses.append(epoch_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_accuracy = val_correct / len(val_dataset) * 100
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}, Loss: {train_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Learning rate scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
        # Save the model with the best validation accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Saved model with validation accuracy {best_val_accuracy:.2f}%")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

# Load the best model (with weights_only=True to avoid security warning)
model.load_state_dict(torch.load(model_save_path, weights_only=True))

# Testing Phase
model.eval()
test_correct = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == targets).sum().item()

test_accuracy = test_correct / len(test_dataset) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Optionally, plot the training and validation losses/accuracies
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.datasets import make_classification
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

# Enable anomaly detection for debugging
torch.autograd.set_detect_anomaly(True)

# Define Mixup function
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Define the Advanced Neural Network without in-place operations
class AdvancedNN(nn.Module):
    def __init__(self):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(5, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 24)
        self.fc4 = nn.Linear(24, 12)
        self.fc5 = nn.Linear(12, 2)
        self.ln1 = nn.LayerNorm(48)
        self.ln2 = nn.LayerNorm(24)
        self.ln3 = nn.LayerNorm(12)
        self.dropconnect = nn.Dropout(p=0.3)

    def forward(self, x):
        x = torch.relu(self.ln1(self.fc1(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x = torch.relu(self.ln2(self.fc2(x)))  # Non-in-place ReLU
        x = self.dropconnect(x)
        x_res = x  # Skip connection
        x = torch.relu(self.fc3(x))  # Non-in-place ReLU for skip connection
        x = x + x_res  # Add skip connection
        x = torch.relu(self.ln3(self.fc4(x)))  # Non-in-place ReLU
        x = self.fc5(x)
        return x

# Hyperparameters and configurations
batch_size = 32
learning_rate = 0.001
num_epochs = 20
patience = 3  # Early stopping patience
accumulation_steps = 2
model_save_path = 'best_model.pth'  # Path to save the best model

# Create a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, optimizer, and learning rate scheduler
model = AdvancedNN()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Apply label smoothing
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, verbose=True)

# Training and Validation Loop with Early Stopping and Gradient Accumulation
best_val_accuracy = 0.0
early_stopping_counter = 0
train_losses = []
val_accuracies = []

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0

    # Track loss over training iterations for smoothing
    for i, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss = loss / accumulation_steps  # Adjust loss for accumulation
        loss.backward()  # Accumulate gradients

        # Update weights every `accumulation_steps` batches
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()

    # Track average training loss for the epoch
    train_losses.append(epoch_loss / len(train_loader))

    # Validation phase
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == targets).sum().item()

    val_accuracy = val_correct / len(val_dataset) * 100
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch}, Loss: {train_losses[-1]:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Learning rate scheduler step
    scheduler.step(val_accuracy)

    # Early stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
        # Save the model with the best validation accuracy
        torch.save(model.state_dict(), model_save_path)
        print(f"Saved model with validation accuracy {best_val_accuracy:.2f}%")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

# Load the best model (with weights_only=True to avoid security warning)
model.load_state_dict(torch.load(model_save_path, weights_only=True))

# Testing Phase
model.eval()
test_correct = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == targets).sum().item()

test_accuracy = test_correct / len(test_dataset) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Optionally, plot the training and validation losses/accuracies
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss / Accuracy')
plt.legend()
plt.show()

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm
import numpy as np

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.targets[idx]

        # If there's a transformation, apply it
        if self.transform:
            image = self.transform(image)

        return image, label

# Define the transformation pipeline: only normalize since data is already tensor
transform = transforms.Compose([
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize for RGB images (mean, std)
])

# Dummy data and targets for demonstration (replace with your actual data)
# Let's assume the images are 32x32 RGB images
data = np.random.rand(100, 3, 32, 32)  # 100 RGB images of size 32x32
targets = np.random.randint(0, 10, size=(100,))  # 100 random class labels (e.g., 10 classes)

# Convert numpy arrays to torch tensors
data = torch.tensor(data, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.long)

# Create the dataset and dataloaders
dataset = CustomDataset(data, targets, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the model (simple CNN example)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)  # Adjust this based on input size after pooling
        self.fc2 = nn.Linear(256, 10)  # 10 classes for classification
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)  # Flatten the tensor for fully connected layer
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        return x

# Initialize model, criterion, and optimizer
model = SimpleCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in tqdm(train_loader, desc="Training"):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt

# Data Preprocessing: Set up augmentations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match VGG input size
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # VGG pre-trained normalization
])

# Load your dataset (replace this with your dataset path and use appropriate Dataset class if needed)
train_dataset = datasets.FakeData(transform=transform)  # Example: Use your actual dataset here
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Load pre-trained VGG16 model and modify the classifier
model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1)
model.classifier[6] = nn.Linear(in_features=4096, out_features=10)  # Modify for 10 classes

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Use Adam optimizer with a lower learning rate

# Training function
def train(model, criterion, optimizer, train_loader, epochs=50):
    train_losses = []
    train_accuracies = []

    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        # Iterate through the training data
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # Update weights
            optimizer.step()

            # Track loss and accuracy
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_preds += labels.size(0)
            correct_preds += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = (correct_preds / total_preds) * 100

        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    return train_losses, train_accuracies

# Train the model
train_losses, train_accuracies = train(model, criterion, optimizer, train_loader, epochs=50)

# Plot training loss and accuracy
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss vs Epochs')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Training Accuracy vs Epochs')
plt.legend()

plt.tight_layout()
plt.show()

# Save the trained model
torch.save(model.state_dict(), 'vgg16_finetuned.pth')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.amp import GradScaler, autocast

# Data Preprocessing: Set up augmentations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match VGG input size
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomRotation(10),  # Random rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # VGG pre-trained normalization
])

# Load your dataset (replace this with your dataset path and appropriate Dataset class if needed)
train_dataset = datasets.FakeData(transform=transform)  # Example: Use your actual dataset here
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Increased batch size

# Load pre-trained VGG16 model and modify the classifier
model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1)
model.classifier[6] = nn.Linear(in_features=4096, out_features=10)  # Modify for 10 classes

# Freeze initial layers to fine-tune only classifier layers initially
for param in model.features.parameters():
    param.requires_grad = False

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)  # Scheduler to reduce learning rate

# Mixed precision training scaler
scaler = GradScaler()

# TensorBoard writer for logging
writer = SummaryWriter(log_dir="runs/vgg16_finetuning")

# Training function
def train(model, criterion, optimizer, train_loader, scheduler, scaler, epochs=50):
    train_losses = []
    train_accuracies = []

    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        # Iterate through the training data
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Mixed precision training: forward pass with autocast
            with autocast(device_type='cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            # Backward pass and gradient scaling
            scaler.scale(loss).backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Step optimizer with scaler
            scaler.step(optimizer)
            scaler.update()

            # Track loss and accuracy
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_preds += labels.size(0)
            correct_preds += (predicted == labels).sum().item()

        # Scheduler step at the end of each epoch
        scheduler.step()

        # Log epoch loss and accuracy
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = (correct_preds / total_preds) * 100
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)

        # TensorBoard logging
        writer.add_scalar("Loss/train", epoch_loss, epoch)
        writer.add_scalar("Accuracy/train", epoch_accuracy, epoch)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        # Unfreeze layers after 10 epochs for full fine-tuning
        if epoch == 9:
            for param in model.features.parameters():
                param.requires_grad = True
            optimizer = optim.AdamW(model.parameters(), lr=0.00001)  # Adjust learning rate for full fine-tuning

    return train_losses, train_accuracies

# Train the model
train_losses, train_accuracies = train(model, criterion, optimizer, train_loader, scheduler, scaler, epochs=50)

# Plot training loss and accuracy
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss vs Epochs')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Training Accuracy vs Epochs')
plt.legend()

plt.tight_layout()
plt.show()

# Save the trained model
torch.save(model.state_dict(), 'vgg16_finetuned.pth')

# Close TensorBoard writer
writer.close()